In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import optimize as op

In [2]:
# 初始化两个空的DataFrame
df_power = pd.DataFrame()
with open('densenet_power_result.txt', 'r') as file:
    lines = file.readlines()

# 初始化变量
frequency = None
batch_size = None

for line in lines:
    if 'frequency' in line:
        frequency, batch_size = line.strip().split(',')
        frequency = int(frequency.split(':')[1])
        batch_size = int(batch_size.split(':')[1])
    elif 'avgpower' in line:
        avgtime = float(line.strip().split(':')[1].replace('w', ''))
        df_power.loc[frequency, batch_size] = avgtime
df_power=df_power.T
print(df_power)
# # 输出结果
df_power.to_csv("densenet_power_results.csv")

         1597      1537      1477      1417      1357      1297      1237  \
256  159.0957  153.1248  149.1499  140.0255  128.1415  120.9257  109.8454   
128  150.3889  140.9989  126.6414  117.0215  109.3551  100.3349  101.1077   
64   115.6323  103.4256   97.1386   90.2726   86.0053   81.7500   85.7541   
32    79.8531   84.9105   72.6010   65.0894   64.0383   62.1635   59.1575   
16    68.5314   64.1915   58.9211   55.7903   53.8025   49.9384   47.5970   
8     62.8917   58.3024   56.1906   52.7524   49.5858   46.4296   42.2074   
4     57.1113   54.7283   49.2113   45.4950   44.6825   42.1125   39.4536   
2     55.7153   51.6283   47.9955   44.2532   41.3827   39.4573   37.7680   
1     55.3594   50.4745   46.9234   44.6251   41.8205   38.7708   37.0317   

         1177      1117     1057  ...     997      967      937      877   \
256  103.0863  100.4837  92.9466  ...  91.7305  88.9500  87.8066  82.0372   
128   90.6939   87.7439  83.2509  ...  82.4645  83.0382  78.7694  75.3453  

In [3]:
data=pd.read_csv('densenet_power_results.csv')
x=np.array(data.iloc[:,0])
data=data.drop(['Unnamed: 0'],axis=1)
data.index=x
data=data.reindex(x)
print(data)

         1597      1537      1477      1417      1357      1297      1237  \
256  159.0957  153.1248  149.1499  140.0255  128.1415  120.9257  109.8454   
128  150.3889  140.9989  126.6414  117.0215  109.3551  100.3349  101.1077   
64   115.6323  103.4256   97.1386   90.2726   86.0053   81.7500   85.7541   
32    79.8531   84.9105   72.6010   65.0894   64.0383   62.1635   59.1575   
16    68.5314   64.1915   58.9211   55.7903   53.8025   49.9384   47.5970   
8     62.8917   58.3024   56.1906   52.7524   49.5858   46.4296   42.2074   
4     57.1113   54.7283   49.2113   45.4950   44.6825   42.1125   39.4536   
2     55.7153   51.6283   47.9955   44.2532   41.3827   39.4573   37.7680   
1     55.3594   50.4745   46.9234   44.6251   41.8205   38.7708   37.0317   

         1177      1117     1057  ...      997      967      937      877  \
256  103.0863  100.4837  92.9466  ...  91.7305  88.9500  87.8066  82.0372   
128   90.6939   87.7439  83.2509  ...  82.4645  83.0382  78.7694  75.3453  

In [4]:
x = np.array(data.index)
y = np.array(data.columns)
print(x)
print(y)
X = []
Y = []
XY = []
Z = []
for i in x:
    for j in y:
        if data.at[i,j]!=np.NaN:
            X.append(i)
            Y.append(int(j))
            XY.append([i,int(j)])
            Z.append(data.at[i,j]) 
print(len(X))
print(len(Y))
print(len(XY))
print(len(Z))

[256 128  64  32  16   8   4   2   1]
['1597' '1537' '1477' '1417' '1357' '1297' '1237' '1177' '1117' '1057'
 '1027' '997' '967' '937' '877' '817' '757' '697' '637' '577' '517']
189
189
189
189


In [5]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,  cross_val_score 
from sklearn.ensemble import GradientBoostingRegressor
import joblib

In [6]:
X_train, X_test, y_train, y_test = train_test_split(XY, Z, test_size=0.3, random_state=112)
model=GradientBoostingRegressor(learning_rate=0.15,min_samples_split=2,max_depth=5,n_estimators=200, criterion ='squared_error',random_state=4)
pre = model.fit(X_train,y_train)

In [7]:
#train 
y_train_pre = model.predict(X_train)

print("Score : "+str(pre.score(X_train,y_train)))
print("MSE : "+str(mean_squared_error(y_train,y_train_pre)))
print("R2 : "+str(r2_score(y_train,y_train_pre)))

Score : 0.9999998458437961
MSE : 9.982222945254536e-05
R2 : 0.9999998458437961


In [8]:
#test 
y_test_pre = model.predict(X_test)

print("Score : "+str(pre.score(X_test,y_test)))
print("MSE : "+str(mean_squared_error(y_test,y_test_pre)))
print("R2 : "+str(r2_score(y_test,y_test_pre)))

Score : 0.986982852379872
MSE : 13.670004761939055
R2 : 0.986982852379872


In [9]:
joblib.dump(filename='densenet_power.pkl',value=model)

['densenet_power.pkl']

In [10]:
import time
import joblib
predicttion=joblib.load('densenet_power.pkl')
input=[[26,1117]]
t1=time.time()
for i in range(11):
    for j in range(15):
        result=predicttion.predict(input)
    result=predicttion.predict(input)
t2=time.time()-t1
print(result)
print(t2*1000)

[55.47424071]
53.73835563659668
